In [1]:
!pwd


/data/guanqzha/kkb_notes


In [9]:

import pandas as pd
raw = pd.read_csv('/home/guanqzha/Data/train.csv')

raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2306871 entries, 0 to 2306870
Data columns (total 29 columns):
order_detail_id           int64
order_id                  int64
order_total_num           float64
order_amount              float64
order_total_payment       float64
order_total_discount      float64
order_pay_time            object
order_status              int64
order_count               float64
is_customer_rate          float64
order_detail_status       float64
order_detail_goods_num    float64
order_detail_amount       float64
order_detail_payment      float64
order_detail_discount     float64
customer_province         object
customer_city             object
member_id                 float64
customer_id               int64
customer_gender           float64
member_status             float64
is_member_actived         float64
goods_id                  int64
goods_class_id            int64
goods_price               float64
goods_status              float64
goods_has_discount

In [10]:
raw.head()

,order_detail_id,order_id,order_total_num,order_amount,order_total_payment,order_total_discount,order_pay_time,order_status,order_count,is_customer_rate,...,customer_gender,member_status,is_member_actived,goods_id,goods_class_id,goods_price,goods_status,goods_has_discount,goods_list_time,goods_delist_time
0,1000000,1000000,1.0,239.9,96.9,0.0,2012-11-01 00:10:56,6,1.0,0.0,...,NaN,NaN,NaN,998,998,54.909289,1.0,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07
1,1001530,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1953,1953,45.961352,0.0,1.0,2013-08-28 17:27:50,2013-09-01 00:38:17
2,1001531,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1083,1083,53.035439,1.0,0.0,2014-10-29 18:21:05,2014-11-05 18:21:05
3,1001532,1001328,3.0,180.0,89.7,0.0,2013-08-31 22:06:35,6,1.0,0.0,...,NaN,NaN,NaN,1013,1013,46.046917,1.0,1.0,2014-10-25 11:00:00,2014-11-01 11:00:00
4,1001533,1001329,1.0,159.9,65.9,0.0,2013-08-31 21:33:36,6,1.0,0.0,...,NaN,NaN,NaN,1628,1628,50.722161,1.0,0.0,2014-10-23 15:35:33,2014-10-30 15:35:33


In [13]:
raw.info

<bound method DataFrame.info of          order_detail_id  order_id  order_total_num  order_amount  \
0                1000000   1000000              1.0         239.9   
1                1001530   1001327              2.0         288.0   
2                1001531   1001327              2.0         288.0   
3                1001532   1001328              3.0         180.0   
4                1001533   1001329              1.0         159.9   
...                  ...       ...              ...           ...   
2306866          3685495   3238358              1.0         199.0   
2306867          3685496   3238359              2.0         299.8   
2306868          3685497   3238359              2.0         299.8   
2306869          3685498   3238360              1.0         168.0   
2306870          3685499   3238361              1.0         102.0   

         order_total_payment  order_total_discount       order_pay_time  \
0                       96.9                   0.0  2012-11-01 0

In [11]:
test = pd.read_csv('/home/guanqzha/Data/submission.csv')
test.head()

,customer_id,result
0,1000000,0.0
1,1000014,0.0
2,1000034,0.0
3,1000046,0.0
4,1000048,0.0


In [12]:
test.info

<bound method DataFrame.info of          customer_id  result
0            1000000     0.0
1            1000014     0.0
2            1000034     0.0
3            1000046     0.0
4            1000048     0.0
...              ...     ...
1585981      2826570     0.0
1585982      2826571     0.0
1585983      2826572     0.0
1585984      2826573     0.0
1585985      2826574     0.0

[1585986 rows x 2 columns]>

In [14]:
raw.shape 

(2306871, 29)

In [15]:
raw['goods_id'].value_counts()

# 对Goods_id进行标签编码
raw['goods_id'] = pd.factorize(raw['goods_id'])[0]
raw['goods_id'].value_counts()

0       472943
5       171277
243      91748
73       67783
121      60263
         ...  
947          1
949          1
950          1
1011         1
1036         1
Name: goods_id, Length: 1037, dtype: int64

In [16]:
raw['customer_id'].nunique() # 158万行


1585986

In [17]:
raw['customer_gender'].isnull().sum()

1671081

In [ ]:

import datetime
import numpy as np

# 对数据进行预处理，各种特征提取
def preprocess(raw, train='train'):
    # 对性别为空的情况，进行0填充
    data = pd.DataFrame(raw.groupby('customer_id')['customer_gender'].last().fillna(0))
    # 添加新的列：商品相关（最后一次行为）
    data[['goods_id_last', 'goods_status_last', 'goods_price_last', 'goods_has_discount_last',           'goods_list_time_last', 'goods_delist_time_last']] = raw.groupby('customer_id')['goods_id',             'goods_status', 'goods_price', 'goods_has_discount', 'goods_list_time', 'goods_delist_time'].last()
    data[['order_total_num_last', 'order_amount_last', 'order_total_payment_last', 'order_total_discount_last', 'order_pay_time_last',           'order_status_last', 'order_count_last', 'is_customer_rate_last', 'order_detail_status_last', 'order_detail_goods_num_last',           'order_detail_amount_last', 'order_detail_payment_last', 'order_detail_discount']] = raw.groupby('customer_id')['order_total_num',         'order_amount', 'order_total_payment', 'order_total_discount', 'order_pay_time', 'order_status', 'order_count', 'is_customer_rate',         'order_detail_status', 'order_detail_goods_num', 'order_detail_amount', 'order_detail_payment', 'order_detail_discount'].last()
    # 添加商品原始价格（多种统计字段） goods_price： mean, min, max, std
    data[['goods_price_mean', 'goods_price_min', 'goods_price_max', 'goods_price_std']] =         raw.groupby('customer_id')['goods_price'].agg([
            ('goods_price_mean', 'mean'),
            ('goods_price_min', 'min'),
            ('goods_price_max', 'max'),
            ('goods_price_std', 'std')])
   # 订单实付金额（多种统计字段） order_detail_payment： mean, min, max, std
    data[['order_detail_payment_mean', 'order_detail_payment_min', 'order_detail_payment_max', 'order_detail_payment_std']] =         raw.groupby('customer_id')['order_detail_payment'].agg([
            ('order_detail_payment_mean', 'mean'),
            ('order_detail_payment_min', 'min'),
            ('order_detail_payment_max', 'max'),
            ('order_detail_payment_std', 'std')])
    # 用户购买的订单数量
    data['count'] = raw.groupby('customer_id')['order_id'].nunique()
    # 用户购买的商品数量
    #data['goods_count'] = raw.groupby('customer_id')['goods_id'].nunique()
    # order_total_num
    data['order_total_sum'] = raw.groupby('customer_id')['order_total_num'].sum()
    # 用户所在省份
    data['customer_province'] = raw.groupby('customer_id')['customer_province'].last()
    # 用户所在城市
    data['customer_city'] = raw.groupby('customer_id')['customer_city'].last()
    # 用户是否评价 统计结果（平均，总和）
    data[['is_customer_rate_ratio', 'is_customer_rate_sum']] = raw.groupby('customer_id')['is_customer_rate'].agg([
        ('is_customer_rate_ratio', np.mean),
        ('is_customer_rate_sum', np.sum)
    ])
    # 用户购买的goods数量
    data['order_detail_total_num'] = raw.groupby('customer_id')['order_detail_goods_num'].sum()
    # 商品折扣统计属性（sum, mean）
    data[['goods_has_discount_sum', 'goods_has_discount_mean']] = raw.groupby('customer_id')['goods_has_discount'].agg([
        ('goods_has_discount_sum', np.sum),
        ('goods_has_discount_mean', np.mean)
    ])
    # 订单实付金额 统计属性（sum, mean）
    data[['order_total_payment_sum', 'order_total_payment_mean']] = raw.groupby('customer_id')['order_total_payment'].agg([
        ('order_total_payment_sum', np.sum),
        ('order_total_payment_mean', np.mean)
    ])
    # 时间转换
    def time2multi(x):
        t = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
        # 2013-01-01 的day = 1, weekday = 星期二（2-1=1）
        return pd.Series([t.month, t.day, t.weekday(), t.hour])
    
    # 起始时间 2013-01-01
    t_str = '2013-01-01 00:00:00'
    t = datetime.datetime.strptime(t_str, '%Y-%m-%d %H:%M:%S')
    
    # 商品最新上架时间diff（假设起始时间为2013-01-01 00:00:00）
    data['goods_list_time_diff'] = data['goods_list_time_last'].map(lambda x: (datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') - t).days/365)
    # 商品最新下架时间diff（假设起始时间为2013-01-01 00:00:00）
    data['goods_delist_time_diff'] = data['goods_delist_time_last'].map(lambda x: (datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') - t).days/365)
    # 商品展示时间
    data['goods_diff'] = data['goods_delist_time_diff'] - data['goods_list_time_diff']
    
    # 付款时间的尺度
    data[['order_pay_time_last_m', 'order_pay_time_last_d', 'order_pay_time_last_week', 'order_pay_time_last_h']] =         data['order_pay_time_last'].apply(time2multi)
    data['order_pay_time_last_diff'] = data['order_pay_time_last'].map(lambda x: (datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') - t).days/365)
    return data


# In[46]:


get_ipython().run_cell_magic('time', '', "# 8月之前的做训练集，8月的数据做验证集\nraw['order_pay_time'].max()\ntrain_raw = raw[raw['order_pay_time'] <= '2013-07-31 23:59:59']\n# 提取训练集 各种特征\ntrain_raw = preprocess(train_raw)\ntrain_raw")


# In[47]:


label_raw = set(raw[raw['order_pay_time'] > '2013-07-31 23:59:59']['customer_id'].dropna())
# 如果这个用户在8月份进行了购买 label=1，否则label=0
# 只有用户有order 就会有行为
train_raw['label'] = train_raw.index.map(lambda x:int(x in label_raw))


# In[48]:


get_ipython().run_cell_magic('time', '', '# 用全量的用户特征进行预测，raw 即9月份之前的所有数据\ntest = preprocess(raw)')


# In[50]:


import pickle
with open('test.pkl', 'wb') as file:
    pickle.dump(test, file)
with open('train_raw.pkl', 'wb') as file:
    pickle.dump(train_raw, file)


# In[53]:


# 有些时间戳提取了时间尺度，需要去掉
train_data = train_raw.drop(['goods_list_time_last', 'goods_delist_time_last', 'order_pay_time_last'], axis=1)


# In[57]:


train_data['customer_province'].fillna('0', inplace=True)
train_data['customer_city'].fillna('0', inplace=True)


# In[76]:


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['customer_province'] = le.fit_transform(train_data['customer_province'])
train_data['customer_city'] = le.fit_transform(train_data['customer_city'])
train_data = train_data.drop(['customer_province', 'customer_city'], axis=1)


# In[77]:


# 分类变量
cate1 = ['goods_id_last', 'goods_status_last', 'order_status_last',         'customer_gender', 'order_detail_status_last',         'order_pay_time_last_h', 'order_pay_time_last_week', 'order_pay_time_last_h']


# In[73]:


#train_data[['customer_province', 'customer_city']].isnull().sum()
train_data['customer_province'] = train_data['customer_province'].astype('int64')
train_data['customer_city'] = train_data['customer_city'].astype('int64')


# In[78]:


train_data.info()


# In[79]:


# 训练集切分
from sklearn.model_selection import KFold
import lightgbm as lgb

# 采用CV=5折交叉验证
kf = KFold(n_splits=5,shuffle=True)
y_pred = 0
for train_index , test_index in kf.split(train_data):
    # 设置每一折的 train和test
    X_train, X_valid, y_train, y_valid = train_data.drop('label', axis=1).iloc[train_index], train_data.drop('label', axis=1).iloc[test_index],         np.array(train_data['label'])[train_index], np.array(train_data['label'])[test_index]
    param = {
        'num_leaves': 121,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'learning_rate': 0.05,
        'metric': 'binary_logloss'
    }
    # 使用lgb进行训练
    trn_data = lgb.Dataset(X_train, y_train)
    val_data = lgb.Dataset(X_valid, y_valid)
    lgbm = lgb.train(param, trn_data, valid_sets=[trn_data, val_data], num_boost_round=10000, early_stopping_rounds=100, verbose_eval=50, categorical_feature=cate1)
    # 计算test的预测结果
    test = test[X_train.columns]
    y_pred = y_pred + lgbm.predict(test)*0.2
    


# In[82]:


pd.DataFrame({
    'column': X_train.columns,
    'importance':lgbm.feature_importance()
}).sort_values(by='importance')


# In[88]:


def f(x):
    """
    if x<0.1:
        return 0.1
    if x>0.9:
        return 0.9
    """
    return round(x*1000)/1000

test['result'] = y_pred
test['result'] = test['result'].map(f)
result = pd.DataFrame(test['result'])
result.to_csv('submission5.csv')
result


# In[96]:


temp = result.reset_index()
temp['result'] = temp['result'].map(lambda x: 0 if x<0.5 else 1)
temp
temp = temp.sort_values(by='customer_id')
temp.to_csv('./test2.csv', index=False)